In [ ]:
! pip uninstall -y torch
! pip uninstall -y transformers

In [ ]:
! pip install --quiet torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --extra-index-url https://download.pytorch.org/whl/cu124
! pip install --quiet transformers==4.48.3

! pip install --quiet datasets scikit-learn
! pip install --quiet numpy pandas tqdm

# Ensemble

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import os
from tqdm import tqdm
from collections import Counter

import numpy as np
import pandas as pd

import torch
from datasets import load_dataset
from transformers import ( AutoModelForImageClassification, AutoProcessor )
from transformers import ( ViTHybridForImageClassification, ViTHybridImageProcessor )

import torch.nn.functional as F
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
model_order = [
    "SodaXII/convnextv2-base-1k-224_rice-leaf-disease-augmented-v4_v5_fft",
    "SodaXII/vit-hybrid-base-bit-384_rice-leaf-disease-augmented-v4_v5_fft",
    "SodaXII/vit-base-patch16-224_rice-leaf-disease-augmented-v4_v5_fft",
    "SodaXII/swin-base-patch4-window7-224_rice-leaf-disease-augmented-v4_v5_fft",
    "SodaXII/deit-base-patch16-224_rice-leaf-disease-augmented-v4_v5_fft",
    "SodaXII/dinov2-base_rice-leaf-disease-augmented-v4_v5_fft",

    "SodaXII/vit_small_patch16_224.augreg_in21k_rice-leaf-disease-augmented-v4_v5_fft",
    "SodaXII/swin-tiny-patch4-window7-224_rice-leaf-disease-augmented-v4_v5_fft",
    "SodaXII/deit-small-patch16-224_rice-leaf-disease-augmented-v4_v5_fft",
    "SodaXII/convnextv2-tiny-1k-224_rice-leaf-disease-augmented-v4_v5_fft",

    "SodaXII/mobilevit-small_rice-leaf-disease-augmented-v4_v5_fft",
    "SodaXII/mobilevitv2_150.cvnets_in22k_ft_in1k_rice-leaf-disease-augmented-v4_v5_fft",
    "SodaXII/efficientnet-b2_rice-leaf-disease-augmented-v4_v5_fft",
    "SodaXII/efficientvit_b1.r224_in1k_rice-leaf-disease-augmented-v4_v5_fft",
    "SodaXII/efficientvit_m4.r224_in1k_rice-leaf-disease-augmented-v4_v5_fft",
    "SodaXII/efficientformer_l1.snap_dist_in1k_rice-leaf-disease-augmented-v4_v5_fft",
    "SodaXII/efficientformerv2_s2.snap_dist_in1k_rice-leaf-disease-augmented-v4_v5_fft",
]

model_name = [
    "ConvNeXtV2 Base",
    "ViT Hybrid Base",
    "ViT Base",
    "Swin Base",
    "DeiT Base",
    "DINOv2 Base",

    "ViT Small",
    "Swin Small",
    "DeiT Small",
    "ConvNeXtV2 Small",

    "MobileViT Small",
    "MobileViTv2 150",
    "EfficientNet B2",
    "EfficientViT B1",
    "EfficientViT M4",
    "EfficientFormer L1",
    "EfficientFormerV2 S2",
]

rename_map = dict(zip(model_order, model_name))

In [ ]:
def load_model_and_processor(model_name, device):
    if "hybrid" in model_name:
        model_cls, proc_cls = ViTHybridForImageClassification, ViTHybridImageProcessor
    else:
        model_cls, proc_cls = AutoModelForImageClassification, AutoProcessor

    model = model_cls.from_pretrained(model_name).to(device).eval()
    processor = proc_cls.from_pretrained(model_name)
    return model, processor

def evaluate_model(model_name, dataset):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, processor = load_model_and_processor(model_name, device)

    y_true, preds, probs = [], [], []
    for ex in tqdm(dataset, desc=f"Evaluating {rename_map[model_name]}"):
        inputs = processor(images=ex["image"], return_tensors="pt").to(device)
        with torch.no_grad():
            logits = model(**inputs).logits.cpu().squeeze(0)

        soft_probs = F.softmax(logits, dim=-1)
        y_true.append(int(ex["label"]))
        preds.append(int(soft_probs.argmax()))
        probs.append(soft_probs.tolist())

    return y_true, preds, probs

In [ ]:
def ensemble_prediction(models, dataset, output_path):
    model_names = [rename_map[m] for m in models]
    all_preds, all_probs, all_accs = {}, {}, {}
    y_true = None

    # evaluate each model
    for model in models:
        y_true, preds, probs = evaluate_model(model, dataset)
        name = rename_map[model]
        all_preds[name], all_probs[name] = preds, probs
        all_accs[name] = accuracy_score(y_true, preds)

    n_models = len(models)
    n_samples = len(y_true)
    labels = sorted(set(y_true))

    # Hard voting
    hard_votes = []
    for i in range(n_samples):
        votes = [all_preds[m][i] for m in model_names]
        hard_votes.append(Counter(votes).most_common(1)[0][0])

    # Soft voting
    soft_votes = []
    for i in range(n_samples):
        avg = np.zeros(len(all_probs[model_names[0]][0]), dtype=float)
        for m in model_names:
            avg += all_probs[m][i]
        avg /= n_models
        soft_votes.append(int(avg.argmax()))

    # choose final predictions
    all_accs['hard_vote'] = accuracy_score(y_true, hard_votes)
    all_accs['soft_vote'] = accuracy_score(y_true, soft_votes)

    # save per-sample votes
    votes_df = pd.DataFrame({m: all_preds[m] for m in model_names})
    votes_df.insert(0, "True", y_true)
    votes_df.insert(len(votes_df.columns), "HardVote", hard_votes)
    votes_df.insert(len(votes_df.columns), "SoftVote", soft_votes)

    # save accuracies
    acc_df = pd.DataFrame({"Model": list(all_accs.keys()), "Accuracy": list(all_accs.values())})

    # classification report & confusion
    report_h = pd.DataFrame(classification_report(y_true, hard_votes, output_dict=True)).T
    report_s = pd.DataFrame(classification_report(y_true, soft_votes, output_dict=True)).T
    cm_h = pd.DataFrame(confusion_matrix(y_true, hard_votes), index=labels, columns=labels)
    cm_s = pd.DataFrame(confusion_matrix(y_true, soft_votes), index=labels, columns=labels)

    # 8. Write to Excel
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        votes_df.to_excel(writer, sheet_name='Votes', index=False)
        acc_df.to_excel(writer, sheet_name='Accuracy', index=False)
        report_h.to_excel(writer, sheet_name='Report_Hard')
        report_s.to_excel(writer, sheet_name='Report_Soft')
        cm_h.to_excel(writer, sheet_name='Confusion_Hard')
        cm_s.to_excel(writer, sheet_name='Confusion_Soft')

    print(f"\n✅ Ensemble results written to {output_path}")

In [ ]:
models = [
  "SodaXII/mobilevit-small_rice-leaf-disease-augmented-v4_v5_fft",
  "SodaXII/mobilevitv2_150.cvnets_in22k_ft_in1k_rice-leaf-disease-augmented-v4_v5_fft",
  "SodaXII/efficientnet-b2_rice-leaf-disease-augmented-v4_v5_fft",
  "SodaXII/efficientvit_b1.r224_in1k_rice-leaf-disease-augmented-v4_v5_fft",
  "SodaXII/efficientformer_l1.snap_dist_in1k_rice-leaf-disease-augmented-v4_v5_fft",
  "SodaXII/efficientformerv2_s2.snap_dist_in1k_rice-leaf-disease-augmented-v4_v5_fft",
]

dataset = load_dataset("cvmil/rice-leaf-disease-augmented-v4", split="test")
output_dir = "/content/drive/Shareddrives/CS198-Drones/[v5] Results"
output_path = os.path.join(output_dir, "ensemble_result.xlsx")
os.makedirs(output_dir, exist_ok=True)

In [ ]:
ensemble_prediction(models, dataset, output_path)